<a href="https://colab.research.google.com/github/rajlm10/D2L-Torch/blob/main/D2L_SNLI_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l huggingface datasets torchinfo -q 

     |████████████████████████████████| 82 kB 342 kB/s 
     |████████████████████████████████| 362 kB 28.0 MB/s 
     |████████████████████████████████| 9.9 MB 85.6 MB/s 
     |████████████████████████████████| 15.7 MB 71.4 MB/s 
     |████████████████████████████████| 11.2 MB 75.1 MB/s 
     |████████████████████████████████| 61 kB 9.2 MB/s 
     |████████████████████████████████| 930 kB 64.6 MB/s 
     |████████████████████████████████| 140 kB 86.3 MB/s 
     |████████████████████████████████| 1.1 MB 37.5 MB/s 
     |████████████████████████████████| 212 kB 55.6 MB/s 
     |████████████████████████████████| 101 kB 10.8 MB/s 
     |████████████████████████████████| 596 kB 84.6 MB/s 
     |████████████████████████████████| 138 kB 57.4 MB/s 
     |████████████████████████████████| 271 kB 86.3 MB/s 
     |████████████████████████████████| 94 kB 4.9 MB/s 
     |████████████████████████████████| 144 kB 92.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all th

In [ ]:
!pip install transformers -q

     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 73.3 MB/s 


In [ ]:
!nvidia-smi

Sat Jun 25 09:06:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import re
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l
import nltk
import collections
import multiprocessing
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torchinfo import summary
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
d2l.DATA_HUB['SNLI'] = ( 'https://nlp.stanford.edu/projects/snli/snli_1.0.zip', '9fcde07509c7e87ec61c640c1b2753d9041758e4')
data_dir = d2l.download_extract('SNLI')

# Creating the Dataset for BORT

In [ ]:
def read_snli(data_dir, is_train):
  """Read the SNLI dataset into premises, hypotheses, and labels.""" 
  def extract_text(s):
    # Remove information that will not be used by us
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    # Substitute two or more consecutive whitespace with space 
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

  label_set = {'entailment': 0, 'contradiction': 1, 'neutral': 2} 
  file_name = os.path.join(data_dir, 'snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt') 
  
  with open(file_name, 'r') as f:
    rows = [row.split('\t') for row in f.readlines()[1:]]
    premises = [extract_text(row[1]) for row in rows if row[0] in label_set] 
    hypotheses = [extract_text(row[2]) for row in rows if row[0] in label_set] 
    labels = [label_set[row[0]] for row in rows if row[0] in label_set]
  return premises, hypotheses, labels

In [ ]:
train_data = read_snli(data_dir, is_train=True)
train_dataset= pd.DataFrame({'premises':train_data[0],'hypotheses':train_data[1],'labels':train_data[2]})
train_dataset.head()

,premises,hypotheses,labels
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competiti...,2
1,A person on a horse jumps over a broken down a...,"A person is at a diner , ordering an omelette .",1
2,A person on a horse jumps over a broken down a...,"A person is outdoors , on a horse .",0
3,Children smiling and waving at camera,They are smiling at their parents,2
4,Children smiling and waving at camera,There are children present,0


In [ ]:
test_data = read_snli(data_dir, is_train=False)
test_dataset= pd.DataFrame({'premises':test_data[0],'hypotheses':test_data[1],'labels':test_data[2]})

In [ ]:
train_dataset.iloc[4,[0,1]].tolist()

['Children smiling and waving at camera', 'There are children present']

In [ ]:
tokenizer =AutoTokenizer.from_pretrained('amazon/bort')
tokenizer.tokenize(train_dataset.iloc[4,[0,1]].tolist(),add_special_tokens=True)

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

['<s>',
 'Children',
 'Ġsmiling',
 'Ġand',
 'Ġwaving',
 'Ġat',
 'Ġcamera',
 '</s>',
 '</s>',
 'There',
 'Ġare',
 'Ġchildren',
 'Ġpresent',
 '</s>']

In [ ]:
tokenizer([train_dataset.iloc[4,[0,1]].tolist()])

{'input_ids': [[0, 31008, 12382, 8, 20615, 23, 2280, 2, 2, 970, 32, 408, 1455, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
train_dataset.iloc[4][['premises','hypotheses']].tolist()

['Children smiling and waving at camera', 'There are children present']

In [ ]:
class CustomTrainDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get item
        item = self.df.iloc[idx]
        text = item[['premises','hypotheses']].tolist()
        label = item['labels']
        # encode text
        encoding = self.tokenizer([text], max_length=64 , truncation=True, return_tensors="pt")
        # remove batch dimension which the tokenizer automatically adds
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        # add label
        encoding["label"] = torch.tensor(label)
        return encoding

In [ ]:
train_dataset = CustomTrainDataset(df=train_dataset, tokenizer=tokenizer)
collate_fn = DataCollatorWithPadding(tokenizer,return_tensors="pt")
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2048, collate_fn= collate_fn,shuffle=True)
eval_dataset = CustomTrainDataset(df=test_dataset, tokenizer=tokenizer)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=2048, collate_fn= collate_fn,shuffle=False)

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(f'{k}: {v.shape}')
# decode the input_ids of the first example of the batch
print(tokenizer.decode(batch['input_ids'][0].tolist()))

input_ids: torch.Size([2048, 64])
attention_mask: torch.Size([2048, 64])
labels: torch.Size([2048])
<s>A man in a gray hooded sweatshirt is pulling a rope trying to maneuver a crane.</s></s>The rope is rough to the touch</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [ ]:
label2id ={'entailment': 0, 'contradiction': 1, 'neutral': 2}
id2label = {v:k for k,v in label2id.items()}
label2id,id2label

({'contradiction': 1, 'entailment': 0, 'neutral': 2},
 {0: 'entailment', 1: 'contradiction', 2: 'neutral'})

In [ ]:
import torch

model = AutoModelForSequenceClassification.from_pretrained('amazon/bort',num_labels=len(label2id))

# I almost always use a learning rate of 5e-5 when fine-tuning Transformer based models
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=1, factor=0.8
    )
epochs = 5
# put model on GPU, if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



Downloading:   0%|          | 0.00/244M [00:00<?, ?B/s]

Some weights of the model checkpoint at amazon/bort were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at amazon/bort and are newly initial

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [ ]:
summary(model)

Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              51,471,360
│    │    └─Embedding: 3-2                              524,288
│    │    └─Embedding: 3-3                              1,024
│    │    └─LayerNorm: 3-4                              2,048
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             23,108,608
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 1,049,600
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           3,075
Total params: 

In [ ]:
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_dataloader):
        # put batch on device
        batch = {k:v.to(device) for k,v in batch.items()}
        
        # forward pass
        outputs = model(**batch)
        loss = outputs.loss
        
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(eval_dataloader):
            # put batch on device
            batch = {k:v.to(device) for k,v in batch.items()}
            
            # forward pass
            outputs = model(**batch)
            loss = outputs.loss
            
            val_loss += loss.item()
    lr_scheduler.step(val_loss)
                  
    print(f"Validation loss after epoch {epoch}:", val_loss/len(eval_dataloader))

100%|██████████| 269/269 [10:06<00:00,  2.26s/it]


Loss after epoch {epoch}: 0.9300059999675112


100%|██████████| 5/5 [00:09<00:00,  1.97s/it]


Validation loss after epoch {epoch}: 0.795401668548584


100%|██████████| 269/269 [10:07<00:00,  2.26s/it]


...

Validation loss after epoch {epoch}: 0.6776490092277527


100%|██████████| 269/269 [10:08<00:00,  2.26s/it]


Loss after epoch {epoch}: 0.6411786916973866


100%|██████████| 5/5 [00:09<00:00,  1.97s/it]

Validation loss after epoch {epoch}: 0.641434371471405


In [ ]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
  if isinstance(net, nn.Module):
    net.eval() # Set the model to evaluation mode 
    if not device:
      device = next(iter(net.parameters())).device 

  # No. of correct predictions, no. of predictions 
  metric = d2l.Accumulator(2)
  with torch.no_grad():
    for batch in tqdm(eval_dataloader):
      batch = {k:v.to(device) for k,v in batch.items()}
       # forward pass
      outputs = model(**batch)
      logits = outputs.logits 
      metric.add(d2l.accuracy(logits, batch['labels']), batch['labels'].numel())
  return metric[0] / metric[1]

In [ ]:
evaluate_accuracy_gpu(model,eval_dataloader,device)

100%|██████████| 5/5 [00:09<00:00,  1.98s/it]


0.7298452768729642

#Prediction

In [ ]:
model.config.id2label=id2label
model.config.label2id=label2id

In [ ]:
from transformers import pipeline

pipe=pipeline(task="text-classification",model=model,tokenizer=tokenizer,device=0)

In [ ]:
pipe({"text":"He has gone out for a walk","text_pair":"He is driving right now"})

{'label': 'contradiction', 'score': 0.8253137469291687}

In [ ]:
pipe({"text":"He has gone out for a walk","text_pair":"He is ten years old"})

{'label': 'neutral', 'score': 0.8863232135772705}

In [ ]:
pipe({"text":"He has gone out for a walk","text_pair":"He is not driving"})

{'label': 'entailment', 'score': 0.47829389572143555}